## データの読み込み

In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
import os,sys
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [18]:
train = pd.read_csv('C:\\Users\\oguma\\Desktop\\山下研究室\\signate\\広島カープ\\球種部門\\特徴量エンジニアリング\\data\\merge2.csv')
train = train.iloc[:,1:]

label = pd.read_csv('C:\\Users\\oguma\\Desktop\\山下研究室\\signate\\広島カープ\\球種部門\\特徴量エンジニアリング\\data\\label.csv')
label = label.iloc[:,1:]

## 新しいラベル生成

In [19]:
label_2 = []
l = label['球種'].tolist()
for i in range(len(l)):
    if l[i] != 0:
        label_2.append(1)
    else:
        label_2.append(0)

label_2 = pd.DataFrame(label_2)
label_2.columns = ['球種']

## データ分割

In [20]:
x_train,x_test,y_train,y_test = train_test_split(train,label_2,test_size = 0.2,random_state = 13)

## モデル(defolt)

In [21]:
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

def model_check(model):
    model.fit(x_train,y_train)
    pre = model.predict(x_test)
    pre_p = model.predict_proba(x_test)
    pre_p2 = model.predict_proba(x_train)
    y_train_pred = classification_report(y_train,model.predict(x_train))
    y_test_pred = classification_report(y_test,pre)
    
    print('train_log_loss:',log_loss(y_train,pre_p2))
    print('test_log_loss:',log_loss(y_test,pre_p))
    print("""[{model_name} \n Train Accuracy: \n{train}] \n Test Accuracy: \n{test}""".format(model_name = model.__class__.__name__,train=y_train_pred,test=y_test_pred))

    return model,pre_p2,pre_p

In [29]:
params = {'max_depth':15,'n_estimators':100}
model,pre_p2,pre_p = model_check(RandomForestClassifier(n_estimators=100,max_depth=80))

train_log_loss: 0.22406313216422652
test_log_loss: 0.7115776333963078
[RandomForestClassifier 
 Train Accuracy: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     96292
           1       1.00      1.00      1.00    109401

   micro avg       1.00      1.00      1.00    205693
   macro avg       1.00      1.00      1.00    205693
weighted avg       1.00      1.00      1.00    205693
] 
 Test Accuracy: 
              precision    recall  f1-score   support

           0       0.53      0.48      0.50     24104
           1       0.57      0.62      0.60     27320

   micro avg       0.55      0.55      0.55     51424
   macro avg       0.55      0.55      0.55     51424
weighted avg       0.55      0.55      0.55     51424



In [16]:
fti = model.feature_importances_
dic = {}
print('feature_importance')
for i ,feat in enumerate(train.columns):
    print('{0:20s}:{1:>.6f}'.format(feat,fti[i]))
    dic[feat]=fti[i]

feature_importance
得点差                 :0.057140
1回                  :0.005216
2回                  :0.005930
3回                  :0.006781
4回                  :0.007061
5回                  :0.007098
6回                  :0.007383
7回                  :0.007128
8回                  :0.006624
9回                  :0.005694
10回                 :0.002408
11回                 :0.001362
表                   :0.013609
試合内投球数              :0.120850
打席内投球数              :0.054962
投手イニング内投球数          :0.089743
No out              :0.011234
1out                :0.013106
No ball             :0.013573
1ball               :0.015650
2ball               :0.013149
No strike           :0.014323
1strike             :0.016309
一塁走者有無              :0.012865
二塁走者有無              :0.011764
三塁走者有無              :0.009581
右投げ                 :0.009656
右打ち                 :0.011368
投手身長                :0.047148
投手体重                :0.050424
投手年俸                :0.056734
打者身長                :0.056367
打者体重                :

In [17]:
dic2 = sorted(dic.items(),key = lambda x:x[1],reverse = True)
print(dic2)
pd.DataFrame(dic2)

[('試合内投球数', 0.1208498749562308), ('投手イニング内投球数', 0.08974289036202551), ('打者年俸', 0.07069212298958401), ('打者体重', 0.06455889295561415), ('得点差', 0.05714048122635276), ('投手年俸', 0.05673370574027106), ('打者身長', 0.056367159576702026), ('打席内投球数', 0.05496178081954278), ('投手体重', 0.05042370225831311), ('投手身長', 0.047148105576772785), ('捕手年俸', 0.037838617524449766), ('捕手体重', 0.03485854219345206), ('捕手身長', 0.02981342728785121), ('1strike', 0.016308647305428206), ('1ball', 0.0156496756490629), ('No strike', 0.014322577801162507), ('表', 0.013608719642037255), ('No ball', 0.013573224930120443), ('2ball', 0.013149277776964287), ('1out', 0.013106029348518655), ('一塁走者有無', 0.012865143954343405), ('二塁走者有無', 0.011763928925963793), ('右打ち', 0.011367811678697399), ('No out', 0.01123371188264912), ('右投げ', 0.009655935819766107), ('三塁走者有無', 0.009581318561784784), ('6回', 0.007382799530026424), ('7回', 0.00712777658611357), ('5回', 0.007098183142303077), ('4回', 0.007061431219713216), ('3回', 0.006780684052398205), ('8回', 

,0,1
0,試合内投球数,0.120850
1,投手イニング内投球数,0.089743
2,打者年俸,0.070692
3,打者体重,0.064559
4,得点差,0.057140
5,投手年俸,0.056734
6,打者身長,0.056367
7,打席内投球数,0.054962
8,投手体重,0.050424
9,投手身長,0.047148


In [15]:
pd.DataFrame(pre_p2)

,0,1,2,3,4,5,6,7
0,0.261846,0.114684,0.122919,0.128709,0.176932,0.081245,0.040860,0.072805
1,0.135865,0.174511,0.186720,0.147608,0.125918,0.107487,0.027351,0.094540
2,0.217046,0.161517,0.112899,0.054771,0.074160,0.292151,0.024072,0.063384
3,0.126390,0.222457,0.125586,0.115703,0.107794,0.094474,0.103352,0.104244
4,0.320345,0.056104,0.481969,0.039249,0.049068,0.026282,0.000825,0.026158
5,0.119316,0.127972,0.114729,0.103355,0.129238,0.099327,0.107045,0.199019
6,0.170139,0.103236,0.176755,0.092917,0.167480,0.089385,0.105863,0.094225
7,0.323475,0.104675,0.100293,0.047222,0.152786,0.162558,0.057231,0.051760
8,0.103407,0.082170,0.105510,0.278787,0.107281,0.212030,0.049961,0.060854
9,0.205286,0.118235,0.097590,0.111934,0.125299,0.155685,0.054752,0.131219


## randomforest gridsearch

In [16]:
param_grid = {'max_depth':[10],
'n_estimators':[40,50,60,70,80]}

In [51]:
forest_grid = GridSearchCV(estimator=RandomForestClassifier(random_state=13,verbose = 1),param_grid = param_grid,
scoring='neg_log_loss',
cv = 3)

In [52]:
forest_grid.fit(x_train,y_train)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    6.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    6.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    1.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_j

NameError: name 'best_params_' is not defined

In [59]:
best_clf = forest_grid.best_estimator_ #ここにベストパラメータの組み合わせが入っています
print('score: {:.2%}'.format(best_clf.score(x_train, y_train)))
y_pred = forest_grid.predict(x_test)
print('score: {:.2%}'.format(best_clf.score(x_test, y_test)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    2.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
score: 46.83%
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
score: 46.88%
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.5s finished
